### Which player has won the most tournaments in a single month?

In [2]:
import os
from dotenv import load_dotenv, find_dotenv

dotenv_path = find_dotenv()
load_dotenv(dotenv_path)

True

##### SQL Solution
**Step 1** group by matches result with tournament

----


**tournament matches number**

~~~sql
SELECT t.tournament_slug, COUNT(t.match_id) AS matches_count
FROM GameInfoResult AS r
LEFT JOIN MatchTournamentInfo AS t ON t.match_id=r.match_id
GROUP BY t.tournament_slug
ORDER BY matches_count DESC;
~~~

Find wins of players for a tournament
~~~sql
SELECT t.tournament_name, t.tournament_slug, r.start_datetime, r.winner_player_id, p.slug AS player_slug, COUNT(r.winner_player_id) AS win_count
FROM GameInfoResult AS r
LEFT JOIN Tournaments AS t ON t.tournament_id=r.tournament_id
LEFT JOIN Players AS p ON p.id=r.winner_player_id
WHERE r.tournament_id=129164
GROUP BY r.winner_player_id, r.tournament_id, YEAR(r.start_datetime), MONTH(start_datetime)
ORDER BY win_count DESC
~~~

Group by tournament id and winner players win count
~~~sql
SELECT o.tournament_name, o.tournament_id, o.player_slug, o.start_datetime, o.winner_player_id, MAX(o.win_count) AS win_count
FROM(
    SELECT t.tournament_id, t.tournament_name, t.tournament_slug, CONCAT(YEAR(r.start_datetime), '-', MONTH(r.start_datetime)) AS start_datetime, r.winner_player_id, p.slug AS player_slug, COUNT(r.winner_player_id) AS win_count
    FROM GameInfoResult AS r
    LEFT JOIN Tournaments AS t ON t.tournament_id=r.tournament_id
    LEFT JOIN Players AS p ON p.id=r.winner_player_id
    WHERE r.winner_player_id != 0
    GROUP BY r.winner_player_id, r.tournament_id, YEAR(r.start_datetime), MONTH(start_datetime)
    ORDER BY YEAR(r.start_datetime) DESC, MONTH(r.start_datetime) DESC, win_count DESC
) AS o GROUP BY o.start_datetime, o.tournament_id ORDER BY win_count DESC LIMIT 1;
~~~



In [5]:
from tennis_data_analysis.db.mysql import Database
import tabulate as tabulate;

mysql = Database(os.environ.get('DB_HOST'), os.environ.get('DB_USERNAME'), os.environ.get('DB_PASSWORD'), os.environ.get('DB_DATABASE'), os.environ.get('DB_PORT'), True)

sql = """
SELECT o.tournament_name, o.tournament_id, o.start_datetime AS `date`, o.player_slug, o.winner_player_id AS player_id, MAX(o.win_count) AS win_count
FROM(
    SELECT t.tournament_id, t.tournament_name, t.tournament_slug, CONCAT(YEAR(r.start_datetime), '-', MONTH(r.start_datetime)) AS start_datetime, r.winner_player_id, p.slug AS player_slug, COUNT(r.winner_player_id) AS win_count
    FROM GameInfoResult AS r
    LEFT JOIN Tournaments AS t ON t.tournament_id=r.tournament_id
    LEFT JOIN Players AS p ON p.id=r.winner_player_id
    WHERE r.winner_player_id != 0
    GROUP BY r.winner_player_id, r.tournament_id, YEAR(r.start_datetime), MONTH(start_datetime)
    ORDER BY YEAR(r.start_datetime) DESC, MONTH(r.start_datetime) DESC, win_count DESC
) AS o GROUP BY o.start_datetime, o.tournament_id ORDER BY win_count DESC LIMIT 1;
"""
mysql.db.execute(sql)
result = mysql.db.fetchone()

if result:
    result = [[col, result[col]] for col in result]
    print(tabulate.tabulate(result, tablefmt='outline', stralign="left", numalign="center"))
else:
    print("Not Found!")



+-----------------+------------------+
| tournament_name | Rome, Italy      |
| tournament_id   | 129829           |
| date            | 2024-5           |
| player_slug     | zverev-alexander |
| player_id       | 4                |
| win_count       | 6                |
+-----------------+------------------+
